In [1]:
from numpy import *

In [4]:
paths='/Users/luokui/Documents/机器学习实战/机器学习实战及配套代码/machinelearninginaction/Ch10/testSet.txt'

def get_data(paths):
    data = []
    with open(paths,'r') as f:
        fr = f.readlines()
        for line in fr:
            line = line.strip().split('\t')
            temp = [float(i) for i in line]
            data.append(temp)   
    return mat(data)
data = get_data(paths)

In [52]:
def cal_dist(centers,data,k):
    data= mat(data)
    m,n = data.shape
    dists = zeros((m,k))
    for j in range(k):
        for i in range(m):
            dists[i,j] = sqrt(sum(pow((centers-data[i]).A,2)))
    return dists

In [31]:
def chose_center(data,k):
    ids = []
    m,n = data.shape
    indexs =list(arange(m))
    for i in range(k):
        NL = len(indexs)
        index = random.randint(0,NL,1)[0]  # 这个地方需要注意啊 ，
        ids.append(indexs[index])
        indexs.remove(indexs[index])
    centers = data[ids]
    return centers
cen = chose_center(data,5)

In [53]:
dists = cal_dist(cen,data,5)

In [58]:
dists.argmax(0)

array([3, 3, 3, 3, 3])

In [60]:
dists.shape

(80, 5)

In [ ]:

# coding: utf-8
import numpy as np
from numpy import *
import matplotlib.pyplot as plt
import numpy
import pandas
import pandas as pd

class km:
 
    def __init__(self,data,k,iters,early_stop=True):
        self.data = data
        self.k = k 
        self.iters = iters
        self.N,_ = data.shape
        self.early_stop = early_stop
        self.init_center = self.chose_center(k,data) #这里的 self.chose_center 的 self
        self.centers,self.class_label,self.total_dist,self.dist,self.rate= self.km_model(data,k,iters,early_stop)
        
    def chose_center(self,k,data):
        ids = []
        indexs =list(np.arange(N))
        for i in range(k):
            NL = len(indexs)
            index = random.randint(0,NL,1)[0]  # 这个地方需要注意啊 ，
            ids.append(indexs[index])
            indexs.remove(indexs[index])
        centers = data[ids]
        return centers
    
    def km_model(self,data,k,iters,early_stop):
        centers= self.chose_center(k,data) 
        total_dist = np.zeros((iters,1))
        rate = []
        
        for j in range(iters):
            dist=[]
            for i in range(k):
                dist.append(np.sqrt(diag(np.matmul((centers[i]-data),(centers[i]-data).T))))  # np.sqrt(np.sum((x[0]-x[2])**2)) 计算的另外一种方法。

            dist = np.transpose(dist)
            class_label = np.argmin(dist,1)

            for i in range(k):
                pos = class_label==i
                centers[i] = np.mean(x[pos],0) 
                total_dist[j] += np.sum(np.sum(dist[pos,i],0))
            
            if j>0 and j<iters:
                re = (total_dist[j-1]-total_dist[j])/total_dist[j]
                rate.append(re)
                
                if early_stop and re<0.01:
                    break
                    
        return centers,class_label,total_dist,dist,rate
""" 
    def plot(self,data,k,iters,early_stop):
        centers,class_label,*_ = self.km_model(data,k,iters,early_stop)
        fig = plt.subplot(1,1,1)
        for i in range(k):
            pos = class_label==i
            fig.scatter(x[pos,0],x[pos,1])
            fig.scatter(centers[i,0],centers[i,1],color='r',marker='*',linewidths=10)
		plt.show()
"""		
		
class km_evo:
    def __init__(self,data,k,iters,method="kendall"):
        self.data = data
        self.k = k 
        self.iters = iters
        self.N,_ = data.shape
        self.method=method
        self.centers,self.class_label,self.total_dist= self.km_model(data,k,iters)


    def chose_random_k(self,k,data):
        ids = []
        indexs = list(np.arange(self.N))
        for i in range(k):
            NL = len(indexs)
            index = random.randint(0,NL,1)[0]  # 这个地方需要注意啊 ，
            ids.append(indexs[index])
            indexs.remove(indexs[index])
        return ids

    def km_model(self,data,k,iters):
        ids = self.chose_random_k(k,data) # 随机选择K
        
        data = data.T
        
        if isinstance(data,numpy.ndarray):
            data = pd.DataFrame(data)
        elif isinstance(data,pandas.core.frame.DataFrame):
            pass
        else:
            raise KeyError("The data is not fit for this model")
        
        dist = np.array(data.corr(method=self.method))      
        total_dist =  np.zeros((iters,1))
        centers = dist[ids]
        class_label = np.argmax(centers,0)
                 
        for j in range(iters):
            new_centers = []   
            for i in range(k):
                pos = class_label==i
                new_dist = dist[:,pos]
                new_index = np.argmax(sum(new_dist,1))            
                new_centers.append(dist[new_index])      
                max_dist = np.max(sum(new_dist,1))              
                total_dist[j] +=max_dist
            centers = new_centers
            class_label =  np.argmax(centers,0)
              
        return centers,class_label,total_dist        
        
    def plot(self,data,k,iters):
        centers,class_label,*_ = self.km_model(data,k,iters)
        fig = plt.subplot(1,1,1)
        for i in range(k):
            pos = class_label==i
            fig.scatter(x[pos,0],x[pos,1])
            fig.scatter(centers[i,0],centers[i,1],color='r',marker='*',linewidths=10)
        plt.show()
